In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\sejin/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25
prediction 결과의 최초 5개 추출


[Prediction(uid='917', iid='50', r_ui=3.0, est=4.180924079910547, details={'was_impossible': False}),
 Prediction(uid='235', iid='318', r_ui=5.0, est=4.458417837000572, details={'was_impossible': False}),
 Prediction(uid='339', iid='12', r_ui=5.0, est=4.45452451367528, details={'was_impossible': False}),
 Prediction(uid='878', iid='755', r_ui=2.0, est=2.9018841395030988, details={'was_impossible': False}),
 Prediction(uid='618', iid='56', r_ui=4.0, est=3.530357815166064, details={'was_impossible': False})]

In [8]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('917', '50', 4.180924079910547),
 ('235', '318', 4.458417837000572),
 ('339', '12', 4.45452451367528)]

In [10]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [11]:
accuracy.rmse(predictions)

RMSE: 0.9285


0.9285252009741827

In [12]:
import pandas as pd

ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')

ratings.to_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv', index=False, header=False)

In [13]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv', reader=reader)

In [15]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [17]:
import pandas as pd
from surprise import Reader, Dataset

In [18]:
ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [22]:
from surprise.model_selection import cross_validate

In [23]:
ratings = pd.read_csv(r'..\999_동영상 연습예제\9장\ml-latest-small\ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8682  0.8755  0.8685  0.8747  0.8731  0.8720  0.0031  
MAE (testset)     0.6652  0.6717  0.6696  0.6729  0.6734  0.6705  0.0030  
Fit time          3.98    3.74    3.76    3.79    3.74    3.80    0.09    
Test time         0.09    0.09    0.15    0.16    0.09    0.12    0.03    


{'test_rmse': array([0.86820714, 0.87547576, 0.86845683, 0.87473227, 0.87306005]),
 'test_mae': array([0.66516836, 0.67168688, 0.66963189, 0.67286584, 0.67337558]),
 'fit_time': (3.984353542327881,
  3.735041856765747,
  3.7649335861206055,
  3.7908637523651123,
  3.742992877960205),
 'test_time': (0.08974981307983398,
  0.08776116371154785,
  0.15059685707092285,
  0.16158699989318848,
  0.08878254890441895)}

In [24]:
from surprise.model_selection import GridSearchCV

In [25]:
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors' : [50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8776312163395398
{'n_epochs': 20, 'n_factors': 50}


In [26]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [27]:
from surprise.dataset import DatasetAutoFolds

In [28]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

data_folds = DatasetAutoFolds(ratings_file = r'..\999_동영상 연습예제\9장\ml-latest-small\ratings_noh.csv',
                             reader=reader)

trainset = data_folds.build_full_trainset()

In [29]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)